<a href="https://colab.research.google.com/github/TarunChaubey/A/blob/master/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problem Statement**

1.      This task involves analyzing the dataset and accordingly assigning the emotion of that image.
2.      Focus on clean code, well commented, and simultaneously obtaining the best result.
3.      The minimum threshold for test accuracy should be 85%. Check on the loss factor too.




 Dataset Link:- https://www.kaggle.com/ameyamote030/einterface-image-dataset/tasks

In [28]:
!nvidia-smi

Wed Jun  2 18:46:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    30W /  70W |   2736MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd '/content/drive/MyDrive/Kaggle'

/content/drive/MyDrive/Kaggle


In [8]:
!unzip '/content/drive/MyDrive/Kaggle/archive.zip' -d '/content/drive/MyDrive/Kaggle'

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/Kaggle/eINTERFACE_2021_Image/train/Sadness/myvideo_frames1255_jpg.rf.026e82030161aab2d89ce54eeb074c4f.jpg  
  inflating: /content/drive/MyDrive/Kaggle/eINTERFACE_2021_Image/train/Sadness/myvideo_frames1255_jpg.rf.0718e0eaae83ce4b6b189da222586cb4.jpg  
  inflating: /content/drive/MyDrive/Kaggle/eINTERFACE_2021_Image/train/Sadness/myvideo_frames1255_jpg.rf.13a2ffdabefcf657568166bdb83c585c.jpg  
  inflating: /content/drive/MyDrive/Kaggle/eINTERFACE_2021_Image/train/Sadness/myvideo_frames125_jpg.rf.26d597870de97a539b47b790842f9e5f.jpg  
  inflating: /content/drive/MyDrive/Kaggle/eINTERFACE_2021_Image/train/Sadness/myvideo_frames125_jpg.rf.39779e0920961bc39bb53a9509ab956a.jpg  
  inflating: /content/drive/MyDrive/Kaggle/eINTERFACE_2021_Image/train/Sadness/myvideo_frames125_jpg.rf.5db8f783d05db83bf9c0c6ab61eb8c53.jpg  
  inflating: /content/drive/MyDrive/Kaggle/eINTERFACE_2021_Image/train/Sadness/myvideo_f

In [17]:
from tensorflow.keras.applications import EfficientNetB0

In [18]:
EfficienmodeltNet = EfficientNetB0(input_shape=(224,224,3), weights='imagenet', include_top=False)

In [19]:
for layer in EfficienmodeltNet.layers:
    EfficienmodeltNet.trainable = False

In [20]:
folders = glob('/content/drive/MyDrive/Kaggle/eINTERFACE_2021_Image/train/*')

In [21]:
x = Flatten()(EfficienmodeltNet.output)

In [22]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=EfficienmodeltNet.input, outputs=prediction)

In [23]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [30]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Kaggle/eINTERFACE_2021_Image/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 11475 images belonging to 6 classes.


In [31]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Kaggle/eINTERFACE_2021_Image/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1438 images belonging to 6 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
 89/359 [======>.......................] - ETA: 32:17 - loss: 2.6986 - accuracy: 0.1640